In [13]:
import gzip
import matplotlib.pyplot as plt
#matplotlib inline
import numpy as np
import numpy.ma as ma
from numpy import matmul as mul
import scipy as sp
from sklearn import manifold
from sklearn import neighbors
from sklearn import datasets
import struct

In [14]:
# Parse data from zipped mnist files into numpy arrays
# Modified from source: https://gist.github.com/tylerneylon/ce60e8a06e7506ac45788443f7269e40
#
# Input:  filename --> Zipped file to parse
#
# Output: return   --> Numpy array of unint8 data points
#
def read_idx(filename):
    with gzip.open(filename) as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

In [15]:
# Construct an nxm matrix of evenly distributed samples from an input sample set
#
# Inputs: Y    --> Sample matrix
#         n, m --> Landmark matrix dimensions
#
# Output: idx  --> nxm Landmark matrix
#
def find_landmarks(Y, n, m):
    xr = np.linspace(np.min(Y[:,0]), np.max(Y[:,0]), n)
    yr = np.linspace(np.min(Y[:,1]), np.max(Y[:,1]), m)
    xg, yg = np.meshgrid(xr, yr)
    idx = [0]*(n*m)
    for i, x, y in zip(range(n*m), xg.flatten(), yg.flatten()):
        idx[i] = int(np.sum(np.abs(Y-np.array([x,y]))**2, axis=-1).argmin())
    return idx

In [16]:
# Inputs: x --> the basis vector (784 x 1)
#         E --> eta is the matrix of nearest neighbors (k x 784)
# 
# Outputs: C --> the covariance matrix of the nearest neighbors of the vector x 
#
# Iterate theough the nearest neighbors and compute the  
#
def calc_covariance(Eta):
    print("Calculate covariance:")
    #print("E: ", np.shape(Eta))
    
    # Compute the local covariance matrix
    C = np.cov(Eta)

    #print("Local covariance matrix C: ", np.shape(C))

    return C

In [17]:
# Clear non-neighbor weight entries from the weight vector
#
# Inputs:  nbrs --> the list of nearest neighbors
#          wght --> the calculated weight vector
# 
# Outputs: wght --> the adjusted weight vector    
#
def clear_non_neighbors(nbrs, wght):
    print("Clear non-nieghbors:")
    #print("wght: ", np.shape(wght), " - ", wght)
    
    # Create mask vector
    dim = np.shape(wght)[0]
    mask = np.ones(dim)                        # ones mean, discard the value
    #print("Mask: ", np.shape(mask), " - ", mask)

    # Apply zeros to the nearest neighbors so we keep the weights
    dim_nbrs = np.shape(nbrs)[0]
    #print("dim nbrs: ", dim_nbrs)
    for i in range(0, dim_nbrs):
        mask[nbrs[i]] = 0;
        #print("nbr: ", nbrs[0,i])
        
    # mask off the non-neighbor weights
    w = ma.masked_array(wght, mask)
    print(w)
    
    return w

In [18]:
# Scale weight values for nearest neighbors
#
# Inputs:  wght --> the wight vector with non-neighbors blanked out
# 
# Outputs: w    --> the adjusted weight vector    
#
def scale_neighbors(wght):
    sumw = np.sum(wght)
    w = wght / sumw

    return w

In [19]:
# Determine the reconstruction weights for each X
#
# Inputs:  X    --> Data Matrix
#          v    --> Index into the data matrix for the vector we want to construct weights for
#          tree --> nearest neighbors tree
#          k    --> number of nearest neighbors  
# 
# Outputs: W    --> the weight matrix
#
def construct_weight_vector(X, v, nbrs, k):
    # Setup matrices and variables
    D = np.shape(X)[1]                              # get the dimension of X
    print("Construct weight matrix")
    #print("X = ", np.shape(X))
    #print("D = ", D)
    #print("k = ", k)
    print("Image #", v, "(v)")
    #print("Neighbors = ", nbrs)

    # Build a matrix of the nearest neighbors
    # (this is all neighbors of the vector v, so remove all others from X)
    Eta = np.delete(X, v, axis=0) 
    #print("Eta: ", np.shape(Eta))
    
    # Compute local covariance matrix for the vector v in X and all neighbors
    C = calc_covariance(Eta)

    # Solve the linear system with constraint that rows of weights sum to one
    b = np.ones(np.shape(C)[1])  # build the constant vector
    w = np.linalg.solve(C, b)    # compute solution
    
    # Apply first constraint to zero out non-neighbor weights
    w = clear_non_neighbors(nbrs, w)
    
    # Apply second constraint to scale valid neighbors
    w = scale_neighbors(w)
    
    #print("w: ", np.shape(w))
    #print(w)
    
    return w

In [20]:
#find bottom d+1 eigenvectors of M
#  (corresponding to the d+1 smallest eigenvalues) 
#set the qth ROW of Y to be the q+1 smallest eigenvector
#  (discard the bottom eigenvector [1,1,1,1...] with eigenvalue zero)
def compute_embedding_components(W, d):
    print("Compute embedding components:")
    #create sparse matrix M = (I-W)'*(I-W)
    I = np.identity(np.shape(W)[0])
    M = mul((I - W).T, (I - W))

    # Find the bottom d + 1 eigenvectors
    U, S, Vt = np.linalg.svd(M)
    
    print("U: ", np.shape(U))
    print("S: ", np.shape(S))
    print("Vt: ", np.shape(Vt))


In [21]:
# Compute the complete nearest neighbors array
#
# Inputs: X      --> training data
#         n_nbrs --> the number of nearest neighbors to search for
#         n_comp --> the number of components
def compute_nearest_neighbors(X, n_nbrs, n_comp):
    print("Computing nearest neighbors...")
    tree = neighbors.BallTree(X, leaf_size=n_comp)
    dist, ind = tree.query(X, k=n_nbrs)
    print("Complete nearest neighbor index: ", np.shape(ind))
    return dist, ind

In [22]:
# TODO - We only have to implement this function, swap it with the manifold one below
# Seek a low-rank projection on an input matrix
#
# Inputs: X            --> Input matrix to reduce
#         n_neighbors  --> Maximum number of neighbors used for reconstruction
#         n_components --> Maximum number of linearly independent components for reconstruction
#
# Output: Y            --> Reconstructed vectors in a lower rank
#         err          --> (Optional implementation) Error margin of vectors
#
def locally_linear_embedding(X, n_neighbors, n_components):
    
    # Step 1: Find the nearest neighbors at for each sample
    #tree = neighbors.BallTree(X, leaf_size=n_components)
    #dist, ind = tree.query(X[:1], k=n_neighbors)
    dist, ind = compute_nearest_neighbors(X, n_neighbors, n_components)
    # ind = indices of k=n_neighbors nearest neighbors
    # (nearest neighbors are ranked in order)
    # dist = distance k=n_neighbors nearest neighbors
    
    # Step 2: Construct a weight matrix that recreates X from its neighbors
    n = np.shape(X)[0]
    d = np.shape(X)[1]
    print("n: ", n)
    print("d: ", d)
    W = np.zeros([n,n-1])
    print("Weight matrix: ", np.shape(W))

    # Loop through each image and construct its weight matrix 
    rows = np.shape(X)[0]
    for r in range(0, rows - 1):
        print("Row: ",r)
        print("ind: ", ind[r,:])
        w = construct_weight_vector(X, r, ind[r,:], n_neighbors)
        W[r,:] = w[:]
    
    # Step 3: Compute vectors that are reconstructed by weights
    Y = compute_embedding_components(W, 2)
    
    Y = X[:,0:2]   # placeholder
    err = 0.001    # placeholder
    return Y, err

In [23]:
# Extract mnist data from files
raw_train = read_idx("train-images-idx3-ubyte.gz")
train_data = np.reshape(raw_train, (60000, 28*28))
train_label = read_idx("train-labels-idx1-ubyte.gz")

In [ ]:
# Train algorithm and calculate landmark graph
X = train_data[train_label == 8]
#Y, err = manifold.locally_linear_embedding(X, n_neighbors=10, n_components=2)
Y, err = locally_linear_embedding(X, n_neighbors=10, n_components=2)
landmarks = find_landmarks(Y, 5, 5)

Computing nearest neighbors...
Complete nearest neighbor index:  (5851, 10)
n:  5851
d:  784
Weight matrix:  (5851, 5850)
Row:  0
ind:  [   0 2748 2880  507  953  605   53 2749 1731   84]
Construct weight matrix
X =  (5851, 784)
D =  784
k =  10
Image # 0 (v)
Neighbors =  [   0 2748 2880  507  953  605   53 2749 1731   84]
Calculate covariance:
E:  (5850, 784)
Local covariance matrix C:  (5850, 5850)
Clear non-nieghbors:
wght:  (5850,)  -  [-3.33952055e+10 -4.65284645e+10  1.18066884e+10 ...  2.42938127e+11
  2.63671911e+10 -1.04129573e+11]
Mask:  (5850,)  -  [1. 1. 1. ... 1. 1. 1.]
dim nbrs:  10
[-33395205518.59908 -- -- ... -- -- --]
w:  (5850,)
Row:  1
ind:  [   1 2006  405   55 1099 4969 4200 2016 2751  769]
Construct weight matrix
X =  (5851, 784)
D =  784
k =  10
Image # 1 (v)
Neighbors =  [   1 2006  405   55 1099 4969 4200 2016 2751  769]
Calculate covariance:
E:  (5850, 784)
Local covariance matrix C:  (5850, 5850)
Clear non-nieghbors:
wght:  (5850,)  -  [ 9.31261610e+10 -4.91

Local covariance matrix C:  (5850, 5850)
Clear non-nieghbors:
wght:  (5850,)  -  [-1.79880205e+11  4.80000105e+10 -1.14412251e+10 ... -2.00912387e+10
 -2.83012115e+10  1.94993249e+10]
Mask:  (5850,)  -  [1. 1. 1. ... 1. 1. 1.]
dim nbrs:  10
[-- -- -- ... -- -- --]
w:  (5850,)
Row:  16
ind:  [  16 4129 1759 1136 4286  229  384 2190   18 1958]
Construct weight matrix
X =  (5851, 784)
D =  784
k =  10
Image # 16 (v)
Neighbors =  [  16 4129 1759 1136 4286  229  384 2190   18 1958]
Calculate covariance:
E:  (5850, 784)
Local covariance matrix C:  (5850, 5850)
Clear non-nieghbors:
wght:  (5850,)  -  [ 2.71872994e+11 -1.82764853e+12  4.92166101e+12 ... -5.61462912e+12
  3.21124075e+10 -3.73362535e+12]
Mask:  (5850,)  -  [1. 1. 1. ... 1. 1. 1.]
dim nbrs:  10
[-- -- -- ... -- -- --]
w:  (5850,)
Row:  17
ind:  [  17 2479  422 5019 1297 1691 2639 1197 1347 2245]
Construct weight matrix
X =  (5851, 784)
D =  784
k =  10
Image # 17 (v)
Neighbors =  [  17 2479  422 5019 1297 1691 2639 1197 1347 2245

Local covariance matrix C:  (5850, 5850)
Clear non-nieghbors:
wght:  (5850,)  -  [ 3.59994752e+12  2.36601459e+12 -1.46476624e+12 ... -2.75340694e+12
  2.84748797e+12 -1.81842308e+12]
Mask:  (5850,)  -  [1. 1. 1. ... 1. 1. 1.]
dim nbrs:  10
[-- -- -- ... -- -- --]
w:  (5850,)
Row:  32
ind:  [  32 2886  830 2884 4894 4450 4196   83 5419 2907]
Construct weight matrix
X =  (5851, 784)
D =  784
k =  10
Image # 32 (v)
Neighbors =  [  32 2886  830 2884 4894 4450 4196   83 5419 2907]
Calculate covariance:
E:  (5850, 784)
Local covariance matrix C:  (5850, 5850)
Clear non-nieghbors:
wght:  (5850,)  -  [ 4.87126022e+10  1.28111874e+09  1.46807014e+10 ...  4.78408927e+10
 -4.13334838e+10  2.64982237e+10]
Mask:  (5850,)  -  [1. 1. 1. ... 1. 1. 1.]
dim nbrs:  10
[-- -- -- ... -- -- --]
w:  (5850,)
Row:  33
ind:  [  33 3693 3682  390 1317 1249  223  398 5613 5773]
Construct weight matrix
X =  (5851, 784)
D =  784
k =  10
Image # 33 (v)
Neighbors =  [  33 3693 3682  390 1317 1249  223  398 5613 5773

Local covariance matrix C:  (5850, 5850)
Clear non-nieghbors:
wght:  (5850,)  -  [ 6.18020396e+10 -1.02558060e+11 -1.96382390e+10 ...  5.05624151e+10
 -4.42932368e+10  3.85953607e+10]
Mask:  (5850,)  -  [1. 1. 1. ... 1. 1. 1.]
dim nbrs:  10
[-- -- -- ... -- -- --]
w:  (5850,)
Row:  48
ind:  [  48 2906 3444 4608  112 1781 5442 5704 1779 1924]
Construct weight matrix
X =  (5851, 784)
D =  784
k =  10
Image # 48 (v)
Neighbors =  [  48 2906 3444 4608  112 1781 5442 5704 1779 1924]
Calculate covariance:
E:  (5850, 784)
Local covariance matrix C:  (5850, 5850)
Clear non-nieghbors:
wght:  (5850,)  -  [ 3.01304683e+11 -2.28934886e+11 -1.83677050e+11 ...  3.00203645e+11
 -2.89880037e+11  4.02917775e+11]
Mask:  (5850,)  -  [1. 1. 1. ... 1. 1. 1.]
dim nbrs:  10
[-- -- -- ... -- -- --]
w:  (5850,)
Row:  49
ind:  [  49 1247 2145 2560  169 4282 2561  402 5155  269]
Construct weight matrix
X =  (5851, 784)
D =  784
k =  10
Image # 49 (v)
Neighbors =  [  49 1247 2145 2560  169 4282 2561  402 5155  269

Local covariance matrix C:  (5850, 5850)
Clear non-nieghbors:
wght:  (5850,)  -  [ 9.69533473e+10  2.57512484e+10  4.91238424e+10 ...  6.44979810e+10
 -1.70904377e+09 -3.08275782e+10]
Mask:  (5850,)  -  [1. 1. 1. ... 1. 1. 1.]
dim nbrs:  10
[-- -- -- ... -- -- --]
w:  (5850,)
Row:  64
ind:  [  64 1782 2755  815  185 5224 4732 4499 4734 1177]
Construct weight matrix
X =  (5851, 784)
D =  784
k =  10
Image # 64 (v)
Neighbors =  [  64 1782 2755  815  185 5224 4732 4499 4734 1177]
Calculate covariance:
E:  (5850, 784)
Local covariance matrix C:  (5850, 5850)
Clear non-nieghbors:
wght:  (5850,)  -  [ 5.67380582e+10  2.47454665e+10 -5.77174471e+10 ...  1.43792098e+11
  2.31969079e+11  6.30460193e+10]
Mask:  (5850,)  -  [1. 1. 1. ... 1. 1. 1.]
dim nbrs:  10
[-- -- -- ... -- -- --]
w:  (5850,)
Row:  65
ind:  [  65 1791 2290 5395 5469 5692 5782 1161 1848 5470]
Construct weight matrix
X =  (5851, 784)
D =  784
k =  10
Image # 65 (v)
Neighbors =  [  65 1791 2290 5395 5469 5692 5782 1161 1848 5470

Local covariance matrix C:  (5850, 5850)
Clear non-nieghbors:
wght:  (5850,)  -  [ 9.29689514e+10  1.98071811e+11  1.04327623e+11 ... -8.05590675e+10
  1.37117609e+11  1.48385089e+11]
Mask:  (5850,)  -  [1. 1. 1. ... 1. 1. 1.]
dim nbrs:  10
[-- -- -- ... -- -- --]
w:  (5850,)
Row:  80
ind:  [  80 1438 2329  533 2319 4290 2323 1167  209   88]
Construct weight matrix
X =  (5851, 784)
D =  784
k =  10
Image # 80 (v)
Neighbors =  [  80 1438 2329  533 2319 4290 2323 1167  209   88]
Calculate covariance:
E:  (5850, 784)
Local covariance matrix C:  (5850, 5850)
Clear non-nieghbors:
wght:  (5850,)  -  [-1.68595643e+11 -1.63520804e+11  5.72120361e+10 ...  9.53938650e+10
 -1.03862531e+11 -1.68359505e+11]
Mask:  (5850,)  -  [1. 1. 1. ... 1. 1. 1.]
dim nbrs:  10
[-- -- -- ... -- -- --]
w:  (5850,)
Row:  81
ind:  [  81  546   74 4124  138 1161 4228  542   82 1074]
Construct weight matrix
X =  (5851, 784)
D =  784
k =  10
Image # 81 (v)
Neighbors =  [  81  546   74 4124  138 1161 4228  542   82 1074

In [ ]:
# Plot the clustered data with landmarks overlaid
plt.scatter(Y[:,0], Y[:,1])
plt.scatter(Y[landmarks,0], Y[landmarks,1])

# Show the landmark samples in a 5x5 grid
fig = plt.figure(figsize=(15,15))
for i in range(len(landmarks)):
    ax = fig.add_subplot(5, 5, i+1)
    imgplot = ax.imshow(np.reshape(X[landmarks[i]], (28,28)), cmap=plt.cm.get_cmap("Greys"))
    imgplot.set_interpolation("nearest")
plt.show()